In [27]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import librosa
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
import feature_engineering
import read_audio
import params

In [20]:
df = pd.read_pickle(params.PICKLE_DATASET_WITH_FEATURES_PATH)

In [21]:
df = df.groupby('drum_type').head(params.N_SAMPLES_PER_CLASS)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Columns: 120 entries, log_attack_time to drum_type
dtypes: float64(119), object(1)
memory usage: 726.0+ KB


In [23]:
drum_type_labels, unique_labels = pd.factorize(df.drum_type)

In [41]:
df = df.assign(drum_type_labels=drum_type_labels)

In [42]:
df.head()

,log_attack_time,temporal_centroid,lat_tc_ratio,duration,release,crest_factor,log_rms_avg,log_rms_std,log_rms_min,log_rms_max,...,mfcc_10_loudest,mfcc_10_d_avg,mfcc_11_avg,mfcc_11_min,mfcc_11_max,mfcc_11_std,mfcc_11_loudest,mfcc_11_d_avg,drum_type,drum_type_labels
0,-1.333109,0.309009,-4.314148,0.301859,0.23220,1.583860,-0.742126,0.296319,-1.509180,-0.470232,...,73.857117,3.746322,25.085449,-27.566612,71.557610,23.789410,71.557610,4.844017,kick,0
1,-1.935169,0.081168,-23.841563,0.162540,0.13932,1.832476,-0.942577,0.359981,-1.483876,-0.555813,...,21.382751,-4.536325,15.984200,4.583979,26.862152,7.882982,15.986156,-2.981111,kick,0
2,-1.333109,0.318289,-4.188365,0.301859,0.23220,1.557335,-0.711129,0.253500,-1.408140,-0.464733,...,66.590034,2.760983,28.809278,3.563507,64.836502,17.678705,64.836502,1.541621,kick,0
3,-1.935169,0.232761,-8.313984,0.162540,0.13932,1.657486,-0.755831,0.319554,-1.413463,-0.444711,...,55.298573,-6.422678,34.397266,7.609301,68.512512,19.782949,48.280617,-5.217404,kick,0
4,-1.333109,0.260443,-5.118622,0.325079,0.25542,1.624353,-0.809102,0.295946,-1.553890,-0.521550,...,47.236877,-2.967587,25.968449,3.578970,55.889328,15.413385,46.749321,-2.640321,kick,0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Columns: 121 entries, log_attack_time to drum_type_labels
dtypes: float64(119), int64(1), object(1)
memory usage: 732.0+ KB


In [28]:
train_clips_df, val_clips_df = train_test_split(df, random_state=0)

In [36]:
train_df = train_clips_df.drop(labels='drum_type_labels', axis=1)
train_df = train_df.drop(labels='drum_type', axis=1)
train_np = train_df.to_numpy()

In [43]:
test_df = val_clips_df.drop(labels='drum_type_labels', axis=1)
test_df = test_df.drop(labels='drum_type', axis=1)
test_np = test_df.to_numpy()

(192, 119)